# CAS 2020 Python Workshop: Session IV Pandas

## Session Descriptions

Welcome to CAS Python Workshop

| No | Date       |Lead |   Contents  |
|:---|:-----------|:----|:------------------------------------------------------------------|
| 1  |  July 15   | BF  | Python programming basics variables, types, lists, dictionaries, functions, dates, strings, dir, help Simulated transactional data, computing Earned Premium (see 5)
| 2  |  July 22   | SM  | Pandas 1: DataFrame creation and basic data manipulation; make a triangle, make development factors, make an exhibit from the CAS Loss Reserve Database
| 3  |  July 29   | BF  | Pandas 2: data io with external sources: Excel, CSV, markdown, HTML, web; advanced data manipulation: querying, merging, indexes, stack, unstack, pivot-table, tidydata Prem and loss simulated data…
|**4**|**Aug 5**|**SM**| **Pandas 3: Visualization and Reporting plotting plus matplotlib, geopandas, jinja, COVID data, NY Auto data**
| 5  |  Aug 12    | SM  | Simulation modeling, pandas, numpy, scipy.stats, Cat model, ELT, YLT, PML Points
| 6  |  Aug 19    | BF  | Linear regression, lm, glm, sklearn Triangles analysis

## Session IV Agenda: Visualization and Reporting

Revised: **Pandas 3: Visualization and Reporting plotting plus matplotlib, jinja**

**Meta-Goal: thought process and elicidating functionality**

### Today's Modules

* Jupyter and Friends: ecosystem 
* Recall from Session III: CAS Triangle Data and Triangles
* Average Loss Ratio by Year by Line Exhibit 
* Simple plotting: Bar Chart, Line Plot, and Histogram 
* Make a Triangle and Develop 
* PPB - Poor Person's Bootstrap 
* Advanced graphics 
* Jinja templates and automating workflow
* ...with some scattered regular expressions 

Google `geopandas` = mapping, GIS extension to Pandas


## Jupyter and Friends

* **Jupyter** = **Ju**lia, **Pyt**hon and **R**
* Project Jupyter is a nonprofit organization created to "develop open-source software, open-standards, and services for interactive computing across dozens of programming languages."
* **JupyterLab** is the next-generation user interface for Project Jupyter. It offers all the familiar building blocks of the classic Jupyter Notebook (notebook, terminal, text editor, file browser, rich outputs, etc.) in a flexible and powerful user interface. The first stable release was announced on February 20, 2018.
* **Colaboratory** (also known as **Colab**) is a free Jupyter notebook environment that runs in the cloud and stores its notebooks on Google Drive
* **IPython** (Interactive Python) is a command shell for interactive computing in multiple programming languages, originally developed for the Python programming language, that offers introspection, rich media, shell syntax, tab completion, and history

*wikipedia.org*

## Load Helpful Introspection Function 

In [ ]:
def sdir(x, colwidth=80):
    """
    Directory of useful elements, wrapped
    """
    from textwrap import fill
    l = [i for i in dir(x) if i[0] != '_']
    mx = max(map(len, l))
    mx += 2
    fs = f'{{:<{mx:d}s}}'
    l = [fs.format(i) for i in l if i[0] != '_']
    print(fill('\t'.join(l), colwidth))
    
sdir(int, 40)

## Read in CAS Data

Direct from URL!

masterdata version contains trivial adjusments to Brian's dataset

Call something else to avoid re-calling

`df.head().T` often shows more useful information 

In [ ]:
import pandas as pd
url = 'http://www.mynl.com/RPM/masterdata.csv'
df_triangle_0 = pd.read_csv(url)
df_triangle_0.head().T

## Add Lag and Subset: Don't Cheat

Work on a copy 

Virtues of zero based lag...

`filter` to view a subset of columns 

`query` to subset rows, SQL like 

In [ ]:
df_triangle = df_triangle_0.copy()
df_triangle['Lag'] -= 1
df_triangle = df_triangle.query(' AY + Lag <= 1997 ')
df_triangle.filter(regex='AY|Lag|GR|Loss').query('Lag == 9 and  UltIncLoss > 10 ').head(10)

In [ ]:
df_triangle.describe().T

In [ ]:
df_triangle['Line'].unique()

In [ ]:
for c in ['AY', 'DY', 'Lag', 'Line', 'GRName']:
    u = df_triangle[c].unique()
    print(f'{c} has {len(u)} unique values, {"" if len(u)<20 else "starting"}\n {u[:20]}\n')

In [ ]:
x = 123456.789012
print(f'{x}\n{x:.3f}\n{x:0,.1f}\n{x:13.4f}')

## Summarize and Plot

Histograms, scatter plot and bar charts

Avg loss ratio by line 

Largest companies by line

Work with Lag == 9 (ultimate) observations to avoid double counting  

Etc.

In [ ]:
# extract subset 
bit = df_triangle.query('DY == 1997')[['GRName', 'Line', 'AY', 'UltIncLoss', 'EarnedPrem']]

# add loss ratio 
bit['LR'] = bit.UltIncLoss / bit.EarnedPrem

# sort values 
bit = bit.sort_values(['Line', 'EarnedPrem'], ascending=[True, False])

# display 
bit.head(10)

In [ ]:
# summarize 1
bit.groupby('Line')[['UltIncLoss', 'EarnedPrem']].sum() 

In [ ]:
# plot
bit.groupby('Line')[['UltIncLoss', 'EarnedPrem']].sum().plot()

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# kind= scatter, hist, bar, barh, line, etc.
bit.groupby('Line')[['UltIncLoss', 'EarnedPrem']].sum().plot(kind='barh')

## Exhibit and Graph Showing Loss Ratio by Line and Year

In [ ]:
bit.groupby(['AY', 'Line'])[['UltIncLoss', 'EarnedPrem']].sum().head(12)

In [ ]:
g = bit.groupby(['AY', 'Line'])
i = 0
for n, b in g:
    print(n)
    display(b.head())
    i += 1
    if i > 3:
        break

In [ ]:
bit.groupby(['AY', 'Line']).sum().head(12)

In [ ]:
import numpy as np

In [ ]:
bit.groupby(['AY', 'Line']).agg({'UltIncLoss': np.sum, 
                                   'EarnedPrem': np.sum, 'LR': np.mean }).head(12)

In [ ]:
bit.groupby(['AY', 'Line']).apply(lambda x : x.UltIncLoss.sum() / x.EarnedPrem.sum()) 

In [ ]:
b = bit.groupby(['AY', 'Line']).apply(lambda x : x.UltIncLoss.sum() / x.EarnedPrem.sum()).unstack(1)
b

In [ ]:
# add all-lines total 
b['All Lines'] = bit.groupby('AY').apply(lambda x : x.UltIncLoss.sum() / x.EarnedPrem.sum()) 
b

In [ ]:
b.plot()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
sdir(ticker)

In [ ]:
# plots with a few frills 
ax = b.plot(lw=1, figsize=(8,6))
ax.xaxis.set_major_locator(ticker.FixedLocator(range(1988, 1998)))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:.0%}'))
ax.grid(lw=.25)
ax.set(ylabel='Loss Ratio', 
       xlabel='Accident Year', 
       title='Med Mal Bucks Improving\nIndustry Loss Ratio Trend',
       )
ax.lines[-1].set(lw=3, c='C0')
ax.lines[1].set(lw=2)
# call legend last to reflect all your changes
ax.legend(ncol=2, fontsize=8, title='Line of Business');

In [ ]:
b.corr()

## Stylin'

In [ ]:
b.style.format('{:0.3f}')

In [ ]:
b.style.format('{:,.3f}').\
    background_gradient(subset=['All Lines'], cmap='viridis_r').\
    bar(vmin=0.5, vmax=1.2, subset=['Med Mal']).\
    set_caption('An Over-Produced DataFrame')

## Subset to a Company or Group of Companies 

Subset to a single company...how do we find it? 

Regular expressions 

`.str` Method 

World of string methods 

Find `contains` 

In [ ]:
sdir(df_triangle.GRName.str)

In [ ]:
?df_triangle.GRName.str.contains

In [ ]:
bit = df_triangle.loc[df_triangle.GRName.str.contains('state', regex=True)]
bit.head()

In [ ]:
bit.GRName.unique()

In [ ]:
import re

In [ ]:
sdir(re)

In [ ]:
bit2 = df_triangle.loc[df_triangle.GRName.str.contains('state', flags=re.IGNORECASE, regex=True)]
bit2.GRName.unique()

In [ ]:
# regular expressions are great...
# all names starting Am or ending Inc or containting ee or ... 
bit2 = df_triangle.loc[df_triangle.GRName.str.contains('^Am|Inc$|[e]{2}', flags=re.IGNORECASE, regex=True)]
bit2.GRName.unique()

## Pull Out a Single Company

There are six lines, let's try to find a co that writes all six

Check you get a single company 

In [ ]:
# lines by company 
lbc = df_triangle.groupby("GRName").apply(lambda x : len(x.Line.unique()) ).sort_values(ascending=False)
lbc[lbc>=5]

In [ ]:
# pick your favorite
co = 'West Bend Mut Ins Grp' 
co

In [ ]:
df = df_triangle.loc[df_triangle.GRName.str.contains(co)]
df.GRName.unique()

In [ ]:
df.Line.unique()

In [ ]:
df.head(10)

## Subset and Index

In [ ]:
# finally just pull in the three columns of interest
co = 'West Bend Mut Ins Grp' 
df = df_triangle.loc[df_triangle.GRName.str.contains(co)]
df = df[['GRName', 'Line', 'AY', 'Lag', 'PaidLoss', 'CaseIncLoss', 'UltIncLoss', 'EarnedPrem']]
df = df.set_index(['GRName', 'Line', 'AY', 'Lag'])
df.head(10)

##  Reshape into Triangle

The joys of `NaN`'s

Display of data vs. form of data 

```
There should be one-- and preferably only one --obvious way to do it.
```

...doesn't really hold for libraries!

use `stack` and `unstack` vs. `pivot` or `pivot_table`.

Triangles are just awkward, whether you convert to a column or not. 

In [ ]:
df[['PaidLoss']].unstack(3).head(10)

## Add Lagged Loss by Group 

Use `groupby` and `shift`

Easier than the `apply` approach (I came up with) last week 

In [ ]:
df.head(10)

In [ ]:
# lagged loss 
df.iloc[:, :2].groupby(level=[0,1,2]).shift(-1).head(10)

In [ ]:
# link ratios 
df.iloc[:, :2].groupby(level=[0,1,2]).shift(-1).head(10) / df.iloc[:, :2].head(10)

In [ ]:
# BTW, of course
(
    df.iloc[:, :2].groupby(level=[0,1,2]).shift(-1).head(10) / df.iloc[:, :2].head(10)
).reset_index(drop=True).plot()

## Link Ratios for Both Lines

In [ ]:
df[[f'{i}LDF' for i in ['Paid', 'Inc']]] = df.iloc[:, :2].groupby(level=[0,1,2]).shift(-1) / df.iloc[:, :2]
df.head(10)

In [ ]:
# paid LDF triangle 
df.iloc[:, -2].unstack(3).head(10)

In [ ]:
# drop 1997 and development 9
# second drop must come after pivot
df.iloc[:, -2].drop(1997, level=2).unstack(3).drop(9, axis=1).head(9)

In [ ]:
trg = df.iloc[:, -2].drop(1997, level=2).unstack(3).drop(9, axis=1).head(9)
trg

## PPB: Poor Person's Bootstrap

Simulate a link ratio from each development lag and take the product to generate a range and distribution of possible FTUs for the most recent accident year. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
def resample(trg, bins=50):
    """
    make distribution of FTUs by resampling link ratio by development lag
    input df with lag columns and rows AYs
    """
    # initialize 
    kpp = np.array([1.0])
    
    # np.kron computes the outer product of two arrays: all sensible products 
    for i in range(0, trg.shape[1]):
        kpp = np.kron(kpp, trg.iloc[:trg.shape[0]-i, i])

    ult = pd.Series( kpp )
    return ult    

In [ ]:
u = resample(trg)
u.shape

In [ ]:
u.head(10)

In [ ]:
fact = lambda x : x == 0 or x * fact(x-1)
fact(9)

In [ ]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(3000)

In [ ]:
fact(1000)

## Intermission: Coding Golf

Challenge: Create bar chart showing frequency of digits in 1000 factorial.

Use as few lines of code as possible. 



In [ ]:
# Coding Golf: 

import pandas as pd

fact = lambda x : x == 0 or x * fact(x-1)

pd.DataFrame( {'digit': np.arange(0,10), 'f': [str(fact(1000)).count(i) for i in '0123456789']} ). \
    set_index('digit'). \
    plot(kind='bar', rot=0, legend=None). \
    set(title='Frequency of Digits in 1000!');

# Python: Par 3 

## Back to PPB

In [ ]:
# 9! = 362880 as expected 
u.describe()

In [ ]:
u.hist(bins=50, ec='white');

In [ ]:
u.sort_values().reset_index(drop=True).plot()

In [ ]:
bit = df[['PaidLDF', 'IncLDF']].drop(1997, level=2, axis=0).drop(9, level=3, axis=0).unstack(3)
bit.head(18)

## Apply PPB to each line of business

In [ ]:
bit.columns.levels[0]

In [ ]:
# b2 = bit.groupby(level=[0,1]).apply(
#    lambda x : pd.DataFrame({'Paid' : resample(x['PaidLDF']), 'Incurred': resample(x['IncLDF'])}))

b2 = bit.groupby(level=[0,1]).apply(lambda x : pd.DataFrame(
        { i : resample(x[i]) for i in x.columns.levels[0]}))

print(b2.shape, 5*fact(9))
b2.head()

In [ ]:
b3 = b2.unstack(1)
b3.head()

In [ ]:
loss12 = df.query(' Lag == 0 and AY == 1997 ')[['PaidLoss', 'CaseIncLoss']].unstack(1).droplevel(2, axis=0)
loss12

## Indexes: You Don't Always Get What You Want

```
You can't always get what you want, 
but if you try sometimes, 
you might find, 
you get what you need.

Mick Jagger
```

In [ ]:
b3.head() * loss12

Wait, whaaat? Product will matches index and then take product.

No overlapping index values! 

Need product ignoring index.

In [ ]:
b4 = b3 * loss12.to_numpy()

In [ ]:
b4.head().T

In [ ]:
b4d = b4.describe()
b4d.loc['CV'] = b4d.loc['std'] / b4d.loc['mean']
b4d

In [ ]:
f, axs = plt.subplots(3, 2, figsize=(8, 9), constrained_layout=True)
axi = iter(axs.flat)

for l in b4.columns.levels[1]:
    ax = next(axi)
    m = sd = 0
    for a, pi in zip([1,.5], b4.columns.levels[0]):
        m0, sd0 = b4d.loc[['mean', 'std'], (pi, l)]
        m = max(m, m0)
        sd = max(sd, sd0)
        bins = b4[(pi, l)].\
            hist(lw=0.5,ec='white', density=True, bins=25, ax=ax, label=pi, alpha=a)
    ax.set(title=l, xlim=[m-4*sd, m+4*sd])
    if l == b4.columns.levels[1][0]:
        ax.legend(loc='upper right')

# add a figure title and drop the last plot
f.suptitle(b3.index[0][0])
f.axes[-1].remove()

In [ ]:
# adjust the x axis on the Work Comp plot
f.axes[4].set(xlim=((25000, 50000)))
f

In [ ]:
# Set all axes the same 
for ax in f.axes[:-1]:
    ax.set(xlim=[-1000,51000])
# tweak legend
f.axes[0].legend(loc='upper left')
f

## So, What's the Answer for WC?

In [ ]:
%who 

In [ ]:
# strings just concatenate; no problem with line breaks within ( ) 
p12 = df_triangle_0.query(' Line == "Work Comp" and '
                    'GRName == @co and AY==1997 and Lag == 9 ' )['UltIncLoss']
p12

In [ ]:
f.axes[4].axvline(p12.iloc[0], c='k', lw=2, label='Actual Ult')
f.axes[4].set(xlim=[25000, 50000])
f.axes[4].legend()
f

In [ ]:
bit.xs('Work Comp', level=1).stack(0).swaplevel(1,2).sort_index()

In [ ]:
# save figure
f.savefig('filename.pdf')

## Jinja Templates and Automating Workflow

In [ ]:
import jinja2
from IPython.display import display, Markdown

In [ ]:
sdir(jinja2)

In [ ]:
?jinja2.Template

In [ ]:
t = jinja2.Template('''

# {{company}} Link Ratios 

## {{line}}

{{ table }}

{{commentary}}

''')

In [ ]:
%who

In [ ]:
trg

In [ ]:
trg.droplevel((0,1))

In [ ]:
table = trg.droplevel((0,1)).fillna(0).to_markdown(floatfmt='.3f').replace("0.000", '')
print(table)

In [ ]:
md = t.render(company="West Bend", line="Commercial Auto", table=table, commentary="Paid LDFs.")
print(md)

In [ ]:
display(Markdown(md))

## Recap

1. Jupyter and Friends
2. Summarize and Plot
3. Styles
4. Discover and `.str`
5. Subset and Index
6. Reshape and Triangles
7. Lags and Links
8. PPB
9. Golf
10. `apply`
11. Advanced plotting example
12. Jinja templates and automating workflow

## Homework

Try weighted average of LDFs or last 3 or last 5.

Make an exhibit showing EP, IL, FTU, Ult, Actual Ultimate, Error by method.

Which is the best method across all cos and lines? 

Play with the original data. Extract largest n cos by line, fastest growing, lowest loss ratio, lowest loss ratio with premium > a reasonable threshold, etc., 

Make exhibits presentation ready.

Export to Excel.

Remember: `sdir(object)` to see what it will do! Use the online help `?function`.

[Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)

[Matplotlib documentation](https://matplotlib.org/)

